### Detect a Bird

In [1]:
from fastbook import *

First, we write a code to dowload bird images:

In [ ]:
urls = search_images_ddg('bird photos', max_images=1)
len(urls),urls[0]

In [ ]:
dest = Path('bird.jpg') # Define the destination path for the image
if not dest.exists():  
    download_url(urls[0], dest, show_progress=False) #  If destination file doesn't exist, download the first image URL to that path

In [ ]:
im = Image.open(dest) # Open the downloaded image
im.to_thumb(256,256) # Display a thumbnail of the image

In [ ]:
searches = 'forest', 'bird'
path = Path('bird_or_not')

if not path.exists():
    for o in searches:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(f'{o} photos') # Searches for forest photos and bird photos
        download_images(dest, urls=results[:200]) # download images
        resize_images(dest, max_size=400, dest=dest) # Resize images to a maximum size of 400 pixels

In [ ]:
# Remove borken images
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), # Define the types of data: images and categories
    get_items=get_image_files, # Function to get image files from the path
    splitter=RandomSplitter(valid_pct=0.2, seed=42), # Split data into training and validation sets
    get_y=parent_label, # Function to get labels from parent directory names
    item_tfms=[Resize(192, method=squish)] # Resize images to 192x192 pixels using squish method
).dataloaders(path) # Create dataloaders

dls.show_batch(max_n=6) # Show a batch of 6 images from the dataloader

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate) # Create a CNN learner using ResNet18 architecture and error rate as metric
learn.fine_tune(3) # Fine-tune the model for 3 epochs

In [ ]:
is_bird,_,probs = learn.predict(PILImage.create('bird.jpg')) # Predict if the image is a bird and get probabilities
print(f'This is a: {is_bird}. Probability: {probs[1]:.4f}')